# Demo KFP pipeline

Install requirements:

In [ ]:
%%bash

pip install kfp~=1.8.14

Imports:

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import kfp
import kfp.dsl as dsl
from kfp.aws import use_aws_secret
from kfp.v2.dsl import (
    component,
    Input,
    Output,
    Dataset,
    Metrics,
    Artifact,
    Model
)

## 1. Connect to client

The default way of accessing Kubeflow is via port-forward. This enables you to get started quickly without imposing any requirements on your environment. Run the following to port-forward Istio's Ingress-Gateway to local port `8080`:

```sh
kubectl port-forward svc/istio-ingressgateway -n istio-system 8080:80
```

In [ ]:
import re
import requests
from urllib.parse import urlsplit

def get_istio_auth_session(url: str, username: str, password: str) -> dict:
    """
    Determine if the specified URL is secured by Dex and try to obtain a session cookie.
    WARNING: only Dex `staticPasswords` and `LDAP` authentication are currently supported
             (we default default to using `staticPasswords` if both are enabled)

    :param url: Kubeflow server URL, including protocol
    :param username: Dex `staticPasswords` or `LDAP` username
    :param password: Dex `staticPasswords` or `LDAP` password
    :return: auth session information
    """
    # define the default return object
    auth_session = {
        "endpoint_url": url,    # KF endpoint URL
        "redirect_url": None,   # KF redirect URL, if applicable
        "dex_login_url": None,  # Dex login URL (for POST of credentials)
        "is_secured": None,     # True if KF endpoint is secured
        "session_cookie": None  # Resulting session cookies in the form "key1=value1; key2=value2"
    }

    # use a persistent session (for cookies)
    with requests.Session() as s:

        ################
        # Determine if Endpoint is Secured
        ################
        resp = s.get(url, allow_redirects=True)
        if resp.status_code != 200:
            raise RuntimeError(
                f"HTTP status code '{resp.status_code}' for GET against: {url}"
            )

        auth_session["redirect_url"] = resp.url

        # if we were NOT redirected, then the endpoint is UNSECURED
        if len(resp.history) == 0:
            auth_session["is_secured"] = False
            return auth_session
        else:
            auth_session["is_secured"] = True

        ################
        # Get Dex Login URL
        ################
        redirect_url_obj = urlsplit(auth_session["redirect_url"])

        # if we are at `/auth?=xxxx` path, we need to select an auth type
        if re.search(r"/auth$", redirect_url_obj.path):

            #######
            # TIP: choose the default auth type by including ONE of the following
            #######

            # OPTION 1: set "staticPasswords" as default auth type
            redirect_url_obj = redirect_url_obj._replace(
                path=re.sub(r"/auth$", "/auth/local", redirect_url_obj.path)
            )
            # OPTION 2: set "ldap" as default auth type
            # redirect_url_obj = redirect_url_obj._replace(
            #     path=re.sub(r"/auth$", "/auth/ldap", redirect_url_obj.path)
            # )

        # if we are at `/auth/xxxx/login` path, then no further action is needed (we can use it for login POST)
        if re.search(r"/auth/.*/login$", redirect_url_obj.path):
            auth_session["dex_login_url"] = redirect_url_obj.geturl()

        # else, we need to be redirected to the actual login page
        else:
            # this GET should redirect us to the `/auth/xxxx/login` path
            resp = s.get(redirect_url_obj.geturl(), allow_redirects=True)
            if resp.status_code != 200:
                raise RuntimeError(
                    f"HTTP status code '{resp.status_code}' for GET against: {redirect_url_obj.geturl()}"
                )

            # set the login url
            auth_session["dex_login_url"] = resp.url

        ################
        # Attempt Dex Login
        ################
        resp = s.post(
            auth_session["dex_login_url"],
            data={"login": username, "password": password},
            allow_redirects=True
        )
        if len(resp.history) == 0:
            raise RuntimeError(
                f"Login credentials were probably invalid - "
                f"No redirect after POST to: {auth_session['dex_login_url']}"
            )

        # store the session cookies in a "key1=value1; key2=value2" string
        auth_session["session_cookie"] = "; ".join([f"{c.name}={c.value}" for c in s.cookies])

    return auth_session

In [ ]:
import kfp

KUBEFLOW_ENDPOINT = "http://localhost:8080"
KUBEFLOW_USERNAME = "user@example.com"
KUBEFLOW_PASSWORD = "12341234"

auth_session = get_istio_auth_session(
    url=KUBEFLOW_ENDPOINT,
    username=KUBEFLOW_USERNAME,
    password=KUBEFLOW_PASSWORD
)

client = kfp.Client(host=f"{KUBEFLOW_ENDPOINT}/pipeline", cookies=auth_session["session_cookie"])
# print(client.list_experiments())

## 2. Components

There are different ways to define components in KFP. Here, we use the **@component** decorator to define the components as Python function-based components.

The **@component** annotation converts the function into a factory function that creates pipeline steps that execute this function. This example also specifies the base container image to run you component in.

Pull data component:

In [ ]:
@component(
    base_image="python:3.10",
    packages_to_install=["numpy~=1.26.4", "pandas~=1.4.2"],
    output_component_file='components/pull_data_component.yaml',
)
def pull_data(url: str, data: Output[Dataset]):
    """
    Pull data component.
    """
    import pandas as pd

    df = pd.read_csv(url, sep=";")
    df.to_csv(data.path, index=None)

Preprocess component:

In [ ]:
@component(
    base_image="python:3.10",
    packages_to_install=["numpy~=1.26.4", "pandas~=1.4.2", "scikit-learn~=1.0.2"],
    output_component_file='components/preprocess_component.yaml',
)
def preprocess(
    data: Input[Dataset],
    scaler_out: Output[Artifact],
    train_set: Output[Dataset],
    test_set: Output[Dataset],
    target: str = "quality",
):
    """
    Preprocess component.
    """
    import pandas as pd
    import pickle
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler

    data = pd.read_csv(data.path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    scaler = StandardScaler()

    train[train.drop(target, axis=1).columns] = scaler.fit_transform(train.drop(target, axis=1))
    test[test.drop(target, axis=1).columns] = scaler.transform(test.drop(target, axis=1))

    with open(scaler_out.path, 'wb') as fp:
        pickle.dump(scaler, fp, pickle.HIGHEST_PROTOCOL)

    train.to_csv(train_set.path, index=None)
    test.to_csv(test_set.path, index=None)

Train component:

In [ ]:
from typing import NamedTuple

@component(
    base_image="python:3.10",
    packages_to_install=["numpy~=1.26.4", "pandas~=1.4.2", "scikit-learn~=1.0.2", "mlflow~=2.4.1", "boto3~=1.21.0"],
    output_component_file='components/train_component.yaml',
)
def train(
    train_set: Input[Dataset],
    test_set: Input[Dataset],
    saved_model: Output[Model],
    mlflow_experiment_name: str,
    mlflow_tracking_uri: str,
    mlflow_s3_endpoint_url: str,
    model_name: str,
    alpha: float,
    l1_ratio: float,
    target: str = "quality",
) -> NamedTuple("Output", [('storage_uri', str), ('run_id', str),]):
    """
    Train component.
    """
    import numpy as np
    import pandas as pd
    from sklearn.linear_model import ElasticNet
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    import mlflow
    import mlflow.sklearn
    import os
    import logging
    import pickle
    from collections import namedtuple

    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)

    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    os.environ['MLFLOW_S3_ENDPOINT_URL'] = mlflow_s3_endpoint_url

    # load data
    train = pd.read_csv(train_set.path)
    test = pd.read_csv(test_set.path)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop([target], axis=1)
    test_x = test.drop([target], axis=1)
    train_y = train[[target]]
    test_y = test[[target]]

    logger.info(f"Using MLflow tracking URI: {mlflow_tracking_uri}")
    mlflow.set_tracking_uri(mlflow_tracking_uri)

    logger.info(f"Using MLflow experiment: {mlflow_experiment_name}")
    mlflow.set_experiment(mlflow_experiment_name)

    with mlflow.start_run() as run:

        run_id = run.info.run_id
        logger.info(f"Run ID: {run_id}")

        model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)

        logger.info("Fitting model...")
        model.fit(train_x, train_y)

        logger.info("Predicting...")
        predicted_qualities = model.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        logger.info("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        logger.info("  RMSE: %s" % rmse)
        logger.info("  MAE: %s" % mae)
        logger.info("  R2: %s" % r2)

        logger.info("Logging parameters to MLflow")
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        # save model to mlflow
        logger.info("Logging trained model")
        mlflow.sklearn.log_model(
            model,
            model_name,
            registered_model_name="ElasticnetWineModel",
            serialization_format="pickle"
        )

        logger.info("Logging predictions artifact to MLflow")
        np.save("predictions.npy", predicted_qualities)
        mlflow.log_artifact(
        local_path="predictions.npy", artifact_path="predicted_qualities/"
        )

        # save model as KFP artifact
        logging.info(f"Saving model to: {saved_model.path}")
        with open(saved_model.path, 'wb') as fp:
            pickle.dump(model, fp, pickle.HIGHEST_PROTOCOL)

        # prepare output
        output = namedtuple('Output', ['storage_uri', 'run_id'])

        # return str(mlflow.get_artifact_uri())
        return output(mlflow.get_artifact_uri(), run_id)

Evaluate component:

In [ ]:
@component(
    base_image="python:3.10",
    packages_to_install=["numpy", "mlflow~=2.4.1"],
    output_component_file='components/evaluate_component.yaml',
)
def evaluate(
    run_id: str,
    mlflow_tracking_uri: str,
    threshold_metrics: dict
) -> bool:
    """
    Evaluate component: Compares metrics from training with given thresholds.

    Args:
        run_id (string):  MLflow run ID
        mlflow_tracking_uri (string): MLflow tracking URI
        threshold_metrics (dict): Minimum threshold values for each metric
    Returns:
        Bool indicating whether evaluation passed or failed.
    """
    from mlflow.tracking import MlflowClient
    import logging

    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)

    client = MlflowClient(tracking_uri=mlflow_tracking_uri)
    info = client.get_run(run_id)
    training_metrics = info.data.metrics

    logger.info(f"Training metrics: {training_metrics}")

    # compare the evaluation metrics with the defined thresholds
    for key, value in threshold_metrics.items():
        if key not in training_metrics or training_metrics[key] > value:
            logger.error(f"Metric {key} failed. Evaluation not passed!")
            return False
    return True

Deploy model component:

In [ ]:
@component(
    base_image="python:3.9",
    packages_to_install=["kserve==0.11.0"],
    output_component_file='components/deploy_model_component.yaml',
)
def deploy_model(model_name: str, storage_uri: str):
    """
    Deploy the model as an inference service with Kserve.
    """
    import logging
    from kubernetes import client
    from kserve import KServeClient
    from kserve import constants
    from kserve import utils
    from kserve import V1beta1InferenceService
    from kserve import V1beta1InferenceServiceSpec
    from kserve import V1beta1PredictorSpec
    from kserve import V1beta1SKLearnSpec
    from kubernetes.client import V1ResourceRequirements

    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)

    model_uri = f"{storage_uri}/{model_name}"
    logger.info(f"MODEL URI: {model_uri}")

    namespace = utils.get_default_target_namespace()
    kserve_version='v1beta1'
    api_version = constants.KSERVE_GROUP + '/' + kserve_version

    isvc = V1beta1InferenceService(
        api_version = api_version,
        kind = constants.KSERVE_KIND,
        metadata = client.V1ObjectMeta(
            name = model_name,
            namespace = namespace,
            annotations = {'sidecar.istio.io/inject':'false'}
        ),
        spec = V1beta1InferenceServiceSpec(
            predictor=V1beta1PredictorSpec(
                service_account_name="kserve-sa",
                min_replicas=1,
                max_replicas = 1,
                sklearn=V1beta1SKLearnSpec(
                    storage_uri=model_uri,
                    resources=V1ResourceRequirements(
                        requests={"cpu": "100m", "memory": "512Mi"},
                        limits={"cpu": "300m", "memory": "512Mi"}
                    )
                ),
            )
        )
    )
    KServe = KServeClient()
    KServe.create(isvc)

Inference component:

In [ ]:
@component(
    base_image="python:3.9",  # kserve on python 3.10 comes with a dependency that fails to get installed
    packages_to_install=["kserve==0.11.0", "scikit-learn~=1.0.2"],
    output_component_file='components/inference_component.yaml',
)
def inference(
    model_name: str,
    scaler_in: Input[Artifact]
):
    """
    Test inference.
    """
    from kserve import KServeClient
    import requests
    import pickle
    import logging
    from kserve import utils
    from urllib.parse import urlsplit
    import re
    
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    
    def get_istio_auth_session(url: str, username: str, password: str) -> dict:
        """
        Determine if the specified URL is secured by Dex and try to obtain a session cookie.
        WARNING: only Dex `staticPasswords` and `LDAP` authentication are currently supported
                 (we default default to using `staticPasswords` if both are enabled)
    
        :param url: Kubeflow server URL, including protocol
        :param username: Dex `staticPasswords` or `LDAP` username
        :param password: Dex `staticPasswords` or `LDAP` password
        :return: auth session information
        """
        # define the default return object
        auth_session = {
            "endpoint_url": url,    # KF endpoint URL
            "redirect_url": None,   # KF redirect URL, if applicable
            "dex_login_url": None,  # Dex login URL (for POST of credentials)
            "is_secured": None,     # True if KF endpoint is secured
            "session_cookie": None  # Resulting session cookies in the form "key1=value1; key2=value2"
        }
    
        # use a persistent session (for cookies)
        with requests.Session() as s:
    
            ################
            # Determine if Endpoint is Secured
            ################
            resp = s.get(url, allow_redirects=True)
            if resp.status_code != 200:
                raise RuntimeError(
                    f"HTTP status code '{resp.status_code}' for GET against: {url}"
                )
    
            auth_session["redirect_url"] = resp.url
    
            # if we were NOT redirected, then the endpoint is UNSECURED
            if len(resp.history) == 0:
                auth_session["is_secured"] = False
                return auth_session
            else:
                auth_session["is_secured"] = True
    
            ################
            # Get Dex Login URL
            ################
            redirect_url_obj = urlsplit(auth_session["redirect_url"])
    
            # if we are at `/auth?=xxxx` path, we need to select an auth type
            if re.search(r"/auth$", redirect_url_obj.path):
    
                #######
                # TIP: choose the default auth type by including ONE of the following
                #######
    
                # OPTION 1: set "staticPasswords" as default auth type
                redirect_url_obj = redirect_url_obj._replace(
                    path=re.sub(r"/auth$", "/auth/local", redirect_url_obj.path)
                )
                # OPTION 2: set "ldap" as default auth type
                # redirect_url_obj = redirect_url_obj._replace(
                #     path=re.sub(r"/auth$", "/auth/ldap", redirect_url_obj.path)
                # )
    
            # if we are at `/auth/xxxx/login` path, then no further action is needed (we can use it for login POST)
            if re.search(r"/auth/.*/login$", redirect_url_obj.path):
                auth_session["dex_login_url"] = redirect_url_obj.geturl()
    
            # else, we need to be redirected to the actual login page
            else:
                # this GET should redirect us to the `/auth/xxxx/login` path
                resp = s.get(redirect_url_obj.geturl(), allow_redirects=True)
                if resp.status_code != 200:
                    raise RuntimeError(
                        f"HTTP status code '{resp.status_code}' for GET against: {redirect_url_obj.geturl()}"
                    )
    
                # set the login url
                auth_session["dex_login_url"] = resp.url
    
            ################
            # Attempt Dex Login
            ################
            resp = s.post(
                auth_session["dex_login_url"],
                data={"login": username, "password": password},
                allow_redirects=True
            )
            if len(resp.history) == 0:
                raise RuntimeError(
                    f"Login credentials were probably invalid - "
                    f"No redirect after POST to: {auth_session['dex_login_url']}"
                )
    
            # store the session cookies in a "key1=value1; key2=value2" string
            auth_session["session_cookie"] = "; ".join([f"{c.name}={c.value}" for c in s.cookies])
    
        return auth_session
    
    KUBEFLOW_ENDPOINT = "http://istio-ingressgateway.istio-system.svc.cluster.local:80"
    KUBEFLOW_USERNAME = "user@example.com"
    KUBEFLOW_PASSWORD = "12341234"
    
    auth_session = get_istio_auth_session(
    url=KUBEFLOW_ENDPOINT,
    username=KUBEFLOW_USERNAME,
    password=KUBEFLOW_PASSWORD,
    )
    TOKEN = auth_session["session_cookie"].replace("authservice_session=", "")
    print("Token:", TOKEN)

    namespace = utils.get_default_target_namespace()

    input_sample = [[5.6, 0.54, 0.04, 1.7, 0.049, 5, 13, 0.9942, 3.72, 0.58, 11.4],
                    [11.3, 0.34, 0.45, 2, 0.082, 6, 15, 0.9988, 2.94, 0.66, 9.2]]

    logger.info(f"Loading standard scaler from: {scaler_in.path}")
    with open(scaler_in.path, 'rb') as fp:
        scaler = pickle.load(fp)

    logger.info(f"Standardizing sample: {scaler_in.path}")
    input_sample = scaler.transform(input_sample)

    # get inference service
    KServe = KServeClient()

    # wait for deployment to be ready
    KServe.get(model_name, namespace=namespace, watch=True, timeout_seconds=120)

    inference_service = KServe.get(model_name, namespace=namespace)
    logger.info(f"inference_service: {inference_service}")

    is_url = f"http://istio-ingressgateway.istio-system.svc.cluster.local:80/v1/models/{model_name}:predict"
    header = {"Host": f"{model_name}.{namespace}.example.com"}
    
    logger.info(f"\nInference service status:\n{inference_service['status']}")
    logger.info(f"\nInference service URL:\n{is_url}\n")

    inference_input = {
        'instances': input_sample.tolist()
    }
    response = requests.post(
        is_url,
        json=inference_input,
        headers=header,
        cookies={"authservice_session": TOKEN}
        
    )
    if response.status_code != 200:
        raise RuntimeError(f"HTTP status code '{response.status_code}': {response.json()}")
    
    logger.info(f"\nPrediction response:\n{response.json()}\n")

## 3. Pipeline

Pipeline definition:

In [ ]:
@dsl.pipeline(
      name='demo-pipeline',
      description='An example pipeline that performs addition calculations.',
)
def pipeline(
    url: str,
    target: str,
    mlflow_experiment_name: str,
    mlflow_tracking_uri: str,
    mlflow_s3_endpoint_url: str,
    model_name: str,
    alpha: float,
    l1_ratio: float,
    threshold_metrics: dict,
):
    pull_task = pull_data(url=url)

    preprocess_task = preprocess(data=pull_task.outputs["data"])

    train_task = train(
        train_set=preprocess_task.outputs["train_set"],
        test_set=preprocess_task.outputs["test_set"],
        target=target,
        mlflow_experiment_name=mlflow_experiment_name,
        mlflow_tracking_uri=mlflow_tracking_uri,
        mlflow_s3_endpoint_url=mlflow_s3_endpoint_url,
        model_name=model_name,
        alpha=alpha,
        l1_ratio=l1_ratio
    )
    train_task.apply(use_aws_secret(secret_name="aws-secret"))

    evaluate_trask = evaluate(
        run_id=train_task.outputs["run_id"],
        mlflow_tracking_uri=mlflow_tracking_uri,
        threshold_metrics=threshold_metrics
    )

    eval_passed = evaluate_trask.output

    with dsl.Condition(eval_passed == "true"):
        deploy_model_task = deploy_model(
            model_name=model_name,
            storage_uri=train_task.outputs["storage_uri"],
        )

        inference_task = inference(
            model_name=model_name,
            scaler_in=preprocess_task.outputs["scaler_out"]
        )
        inference_task.after(deploy_model_task)

Pipeline arguments:

In [ ]:
# Specify pipeline argument values

eval_threshold_metrics = {'rmse': 0.9, 'r2': 0.3, 'mae': 0.8}

arguments = {
    "url": "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv",
    "target": "quality",
    "mlflow_tracking_uri": "http://mlflow.mlflow.svc.cluster.local:5000",
    "mlflow_s3_endpoint_url": "http://mlflow-minio-service.mlflow.svc.cluster.local:9000",
    "mlflow_experiment_name": "demo-notebook",
    "model_name": "wine-quality",
    "alpha": 0.5,
    "l1_ratio": 0.5,
    "threshold_metrics": eval_threshold_metrics
}

## 4. Submit run

In [ ]:
run_name = "demo-run"
experiment_name = "demo-experiment"

client.create_run_from_pipeline_func(
    pipeline_func=pipeline,
    run_name=run_name,
    experiment_name=experiment_name,
    arguments=arguments,
    mode=kfp.dsl.PipelineExecutionMode.V2_COMPATIBLE,
    enable_caching=False,
    namespace="kubeflow-user-example-com"
)

## 5. Check run

### Kubeflow Pipelines UI

The default way of accessing Kubeflow is via port-forward. This enables you to get started quickly without imposing any requirements on your environment. Run the following to port-forward Istio's Ingress-Gateway to local port `8080`:

```sh
kubectl port-forward svc/istio-ingressgateway -n istio-system 8080:80
```

After running the command, you can access the Kubeflow Central Dashboard by doing the following:

1. Open your browser and visit [http://localhost:8080/](http://localhost:8080/). You should get the Dex login screen.
2. Login with the default user's credential. The default email address is `user@example.com` and the default password is `12341234`.

### MLFlow UI

To access MLFlow UI, open a terminal and forward a local port to MLFlow server:

<br>

```bash
kubectl -n mlflow port-forward svc/mlflow 5000:5000
```

<br>

Now MLFlow's UI should be reachable at [`http://localhost:5000`](http://localhost:5000).

## 6. Check deployed model

```bash
# get inference services
kubectl -n kubeflow-user-example-com get inferenceservice

# get deployed model pods
kubectl -n kubeflow-user-example-com get pods

# delete inference service
kubectl -n kubeflow-user-example-com delete inferenceservice wine-quality
```
<br>

If something goes wrong, check the logs with:

<br>

```bash
kubectl logs -n kubeflow-user-example-com <pod-name> kserve-container

kubectl logs -n kubeflow-user-example-com <pod-name> queue-proxy

kubectl logs -n kubeflow-user-example-com <pod-name> storage-initializer
```


## 7. Troubleshooting

If the inference isn't working, try to patch the knative-serving config-domain:



```bash
kubectl patch cm config-domain --patch '{"data":{"example.com":""}}' -n knative-serving
```